### **اتصال GOOGLE COLAB به GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **فراخوانی کتابخانه های مورد نیاز**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns

### **فراخوانی دیتاست از GOOGLE DRIVE و نمایش آن**

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/loans.csv")
data

### **تبدیل اطلاعات به صورت دیتا فریم**

In [ ]:
df=pd.DataFrame(data)

### **EXPLORATION: اکتشاف کلی در داده ها**

In [ ]:
print(df.head())
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.duplicated().sum())
print(df.nunique())
print(df.info)
print(df.describe())
print(df.isnull().sum())
print(df.corr(numeric_only=True))
print(df['loan_type'].value_counts())
print(df['loan_type'].value_counts().to_frame())
print((df['loan_type'].str.isnumeric()==False).sum())

### **تبدیل داده های رشته ای و چند حالته به داده های عددی(LABEL ENCODING)**

In [ ]:
def labelEncoder(data,columns):
   le=LabelEncoder()
   for col in columns:
    data[col]=le.fit_transform(data[col])+1
   return data


df=labelEncoder(df,['client_id','loan_type'])
print(df)


### **همه اعداد درون ستون "پرداخت شده" را با عدد 1  جمع میکنیم تا مقادیر صفر از بین بروند**

In [ ]:
df['repaid']=df['repaid']+1
df

### **حذف ستونهای اضافه**

In [ ]:
df=df.drop(['loan_start','loan_end','loan_id'],axis=1)
print(df.dtypes)

### **ایجاد ستونهای جدید که ویژگی های جدیدی به داده ها اضافه میکنند**

In [ ]:

df['extra1']=df['loan_amount']*df['loan_type']
df['extra2']=df['loan_amount']*df['repaid']
df['extra3']=df['loan_amount']*df['loan_type']*df['repaid']
df

### **نرمال سازی داده ها**

In [ ]:
def min_max_scaler(data,columns):
   scaler=MinMaxScaler()
   data=scaler.fit_transform(data)
   data=pd.DataFrame(data)
   data.columns=columns
   return data


df=min_max_scaler(df,['client_id','loan_type','loan_amount','repaid','rate','extra1','extra2','extra3'])
df

### **نمایش جدول همبستگی برای نشان دادن میزان همبستگی بین ستونها**

In [ ]:
plt.figure(figsize=(12,8))
corr=df.corr()
mask=np.triu(corr)
sns.heatmap(corr,square=True,annot=True,mask=mask,fmt='.2f')
plt.show()

### **ساخت، آموزش و آزمون مدل به روش رگرسیون خطی تک متغیره**

In [ ]:
X=df[['extra2']]
y=df['loan_amount']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
r2=r2_score(y_test,y_pred)
print(f"R_squared:{r2}")

### **ساخت، آموزش و آزمون مدل به روش رگرسیون خطی چند متغیره**

In [ ]:
features=['extra1','extra2','extra3']
X=df[features]
y=df[['loan_amount']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
r2=r2_score(y_test,y_pred)
print(f"R_squared:{r2}")

  
### **ساخت، آموزش و آزمون مدل به روش رگرسیون چند جمله ای با درجه 2**

In [ ]:
features=['extra1','extra2','extra3']
X=df[features]
y=df[['loan_amount']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)
model=LinearRegression()
pf=PolynomialFeatures(degree=2)
X_train_poly=pf.fit_transform(X_train)
X_test_poly=pf.fit_transform(X_test)
model.fit(X_train_poly,y_train)
print(model.score(X_test_poly,y_test))

  
### **ساخت، آموزش و آزمون مدل به روش رگرسیون چند جمله ای با درجه 3**

In [ ]:
features=['extra1','extra2','extra3']
X=df[features]
y=df[['loan_amount']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)
model=LinearRegression()
pf=PolynomialFeatures(degree=3)
X_train_poly=pf.fit_transform(X_train)
X_test_poly=pf.fit_transform(X_test)
model.fit(X_train_poly,y_train)
print(model.score(X_test_poly,y_test))